# Football Tactics AI: Comprehensive Implementation

This notebook implements a complete football tactics simulation system with:
- Role-specific player behaviors (DEF, MID, FWD)
- Tactical pass types and patterns
- Enhanced physics with interception mechanics
- Match simulation and ML-based tactical recommendations

## Sources and Citations
- StatsBomb Open Data: https://github.com/statsbomb/open-data
- Football pitch dimensions: FIFA regulations (105m x 68m)
- Tactical concepts: Based on established football theory (Tiki-Taka, Counter-Attack, Wing Play)
- Machine Learning: Scikit-learn Random Forest Classifier

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from enum import Enum
import random
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

try:
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report, accuracy_score
    import pickle
except ImportError:
    import subprocess
    subprocess.check_call(['pip', 'install', '-q', 'scikit-learn'])
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report, accuracy_score
    import pickle

print('✓ Libraries loaded')

## 3. Core Data Structures

Define player roles, action types, and core data structures.

In [ ]:
class Role(Enum):
    DEF = 'defender'
    MID = 'midfielder'
    FWD = 'forward'

class ActionType(Enum):
    PASS = 'pass'
    SHOT = 'shot'
    DRIBBLE = 'dribble'
    TACKLE = 'tackle'

class PassType(Enum):
    SHORT = 'short'
    MEDIUM = 'medium'
    LONG = 'long'
    THROUGH = 'through'

class TacticalPattern(Enum):
    TIKI_TAKA = 'tiki_taka'
    COUNTER_ATTACK = 'counter_attack'
    WING_PLAY = 'wing_play'
    DIRECT = 'direct'

@dataclass
class Position:
    x: float
    y: float

@dataclass
class Player:
    id: int
    role: Role
    position: Position
    skill: float

print('✓ Core data structures defined')

## 4. Tactical Pass System

Implement pass characteristics and selection logic.

In [ ]:
@dataclass
class PassCharacteristics:
    min_distance: float
    max_distance: float
    risk_factor: float
    success_base: float

PASS_CONFIGS = {
    PassType.SHORT: PassCharacteristics(1, 15, 0.1, 0.92),
    PassType.MEDIUM: PassCharacteristics(15, 30, 0.3, 0.78),
    PassType.LONG: PassCharacteristics(30, 60, 0.6, 0.58),
    PassType.THROUGH: PassCharacteristics(10, 35, 0.7, 0.45)
}

def select_pass_type(distance: float, role: Role, pattern: TacticalPattern) -> PassType:
    """Select pass type based on distance, role, and tactical pattern."""
    if pattern == TacticalPattern.TIKI_TAKA:
        return PassType.SHORT if distance < 20 else PassType.MEDIUM
    elif pattern == TacticalPattern.COUNTER_ATTACK:
        return PassType.THROUGH if distance > 25 and role != Role.DEF else PassType.LONG
    elif pattern == TacticalPattern.WING_PLAY:
        return PassType.LONG if role == Role.DEF else PassType.MEDIUM
    else:
        if distance < 15:
            return PassType.SHORT
        elif distance < 30:
            return PassType.MEDIUM
        else:
            return PassType.LONG

print('✓ Tactical pass system implemented')

## 5. Tactical Patterns

Implement tactical pattern selection and modifiers.

In [ ]:
def select_tactical_pattern(ball_position: Position, player_role: Role) -> TacticalPattern:
    """Select tactical pattern based on field position and player role."""
    if ball_position.x < 35:
        return TacticalPattern.TIKI_TAKA
    elif ball_position.x > 70:
        if player_role == Role.FWD:
            return TacticalPattern.DIRECT
        return TacticalPattern.WING_PLAY
    else:
        if abs(ball_position.y - 34) > 20:
            return TacticalPattern.WING_PLAY
        return TacticalPattern.COUNTER_ATTACK

def apply_tactical_modifier(base_success: float, pattern: TacticalPattern, 
                           pass_type: PassType, role: Role) -> float:
    """Apply tactical modifiers to base success probability."""
    modifier = 1.0
    
    if pattern == TacticalPattern.TIKI_TAKA:
        if pass_type == PassType.SHORT:
            modifier += 0.15
        if role == Role.MID:
            modifier += 0.08
    elif pattern == TacticalPattern.COUNTER_ATTACK:
        if pass_type == PassType.THROUGH:
            modifier += 0.12
        if role == Role.FWD:
            modifier += 0.10
    elif pattern == TacticalPattern.WING_PLAY:
        if pass_type == PassType.LONG:
            modifier += 0.10
    
    return min(base_success * modifier, 0.98)

print('✓ Tactical patterns implemented')

## 6. Enhanced Physics

Implement interception and pass success calculations.

In [ ]:
def calculate_interception(pass_start: Position, pass_end: Position, 
                          opponents: List[Player]) -> float:
    """Calculate interception probability based on opponent positions."""
    interception_prob = 0.0
    
    for opp in opponents:
        dist_to_line = abs((pass_end.y - pass_start.y) * opp.position.x - 
                          (pass_end.x - pass_start.x) * opp.position.y + 
                          pass_end.x * pass_start.y - pass_end.y * pass_start.x) / \
                      np.sqrt((pass_end.y - pass_start.y)**2 + (pass_end.x - pass_start.x)**2)
        
        if dist_to_line < 3.0:
            interception_prob += (1.0 - dist_to_line / 3.0) * 0.15 * opp.skill
    
    return min(interception_prob, 0.6)

def calculate_pass_success(passer: Player, receiver: Player, pass_type: PassType,
                          pattern: TacticalPattern, opponents: List[Player]) -> bool:
    """Calculate whether a pass succeeds."""
    config = PASS_CONFIGS[pass_type]
    
    base_success = config.success_base * passer.skill
    
    success_prob = apply_tactical_modifier(base_success, pattern, pass_type, passer.role)
    
    interception_prob = calculate_interception(passer.position, receiver.position, opponents)
    
    final_success = success_prob * (1 - interception_prob)
    
    return random.random() < final_success

print('✓ Enhanced physics implemented')

## 7. Team Setup

Create teams with 4-3-3 formation.

In [ ]:
def create_team(team_id: int, formation: str = '433') -> List[Player]:
    """Create a team with specified formation."""
    players = []
    
    if formation == '433':
        positions = [
            (15, 34, Role.DEF), (20, 14, Role.DEF), (20, 54, Role.DEF), (25, 34, Role.DEF),
            (40, 20, Role.MID), (45, 34, Role.MID), (40, 48, Role.MID),
            (70, 14, Role.FWD), (75, 34, Role.FWD), (70, 54, Role.FWD)
        ]
    
    for i, (x, y, role) in enumerate(positions):
        if team_id == 2:
            x = 105 - x
        
        skill = np.random.uniform(0.75, 0.95)
        players.append(Player(
            id=team_id * 100 + i,
            role=role,
            position=Position(x, y),
            skill=skill
        ))
    
    return players

team1 = create_team(1, '433')
team2 = create_team(2, '433')

print(f'✓ Created team 1: {len(team1)} players')
print(f'✓ Created team 2: {len(team2)} players')
print(f'Sample player: ID={team1[0].id}, Role={team1[0].role.value}, Skill={team1[0].skill:.2f}')

## 8. Match Simulation

Implement match simulator with event tracking.

In [ ]:
@dataclass
class MatchEvent:
    time: int
    action: ActionType
    player_role: Role
    position: Position
    pass_type: Optional[PassType]
    pattern: TacticalPattern
    success: bool
    distance: float = 0.0

class MatchSimulator:
    def __init__(self, team1: List[Player], team2: List[Player]):
        self.team1 = team1
        self.team2 = team2
        self.events = []
    
    def simulate_action(self, current_player: Player, teammates: List[Player],
                       opponents: List[Player], time: int):
        """Simulate a single action."""
        pattern = select_tactical_pattern(current_player.position, current_player.role)
        
        action_weights = {
            ActionType.PASS: 0.70,
            ActionType.DRIBBLE: 0.15,
            ActionType.SHOT: 0.10 if current_player.position.x > 85 else 0.02,
            ActionType.TACKLE: 0.05
        }
        
        action = random.choices(
            list(action_weights.keys()),
            weights=list(action_weights.values())
        )[0]
        
        success = False
        pass_type = None
        distance = 0.0
        
        if action == ActionType.PASS:
            receiver = random.choice([p for p in teammates if p.id != current_player.id])
            distance = np.sqrt((receiver.position.x - current_player.position.x)**2 +
                             (receiver.position.y - current_player.position.y)**2)
            pass_type = select_pass_type(distance, current_player.role, pattern)
            success = calculate_pass_success(current_player, receiver, pass_type, pattern, opponents)
        elif action == ActionType.SHOT:
            distance = 105 - current_player.position.x
            success = random.random() < (0.15 * current_player.skill)
        elif action == ActionType.DRIBBLE:
            distance = random.uniform(3, 10)
            success = random.random() < (0.65 * current_player.skill)
        else:
            success = random.random() < (0.55 * current_player.skill)
        
        event = MatchEvent(
            time=time,
            action=action,
            player_role=current_player.role,
            position=Position(current_player.position.x, current_player.position.y),
            pass_type=pass_type,
            pattern=pattern,
            success=success,
            distance=distance
        )
        
        self.events.append(event)
        return success
    
    def simulate_match(self, num_actions: int = 150):
        """Simulate a complete match."""
        possession_team = 1
        
        for i in range(num_actions):
            if possession_team == 1:
                current_player = random.choice(self.team1)
                success = self.simulate_action(current_player, self.team1, self.team2, i)
            else:
                current_player = random.choice(self.team2)
                success = self.simulate_action(current_player, self.team2, self.team1, i)
            
            if not success:
                possession_team = 2 if possession_team == 1 else 1
        
        return self.events

print('✓ Match simulator implemented')

## 9. Data Loading

Attempt to load StatsBomb data (optional).

In [ ]:
# Enhanced data loading with proper player, team, and match data fetching
import json

# Initialize data structures
player_data = {}
team_data = {}
match_data = []

# Load StatsBomb open data for training
try:
    import urllib.request
    
    # Fetch competition data (Premier League)
    competition_url = 'https://raw.githubusercontent.com/statsbomb/open-data/master/data/competitions.json'
    with urllib.request.urlopen(competition_url) as response:
        competitions = json.loads(response.read())
    
    # Get Premier League competition
    premier_league = [c for c in competitions if c.get('competition_name') == 'Premier League']
    if premier_league:
        comp_id = premier_league[0]['competition_id']
        season_id = premier_league[0]['season_id']
        
        # Fetch matches for this competition
        matches_url = f'https://raw.githubusercontent.com/statsbomb/open-data/master/data/matches/{comp_id}/{season_id}.json'
        with urllib.request.urlopen(matches_url) as response:
            matches = json.loads(response.read())
        
        print(f'✓ Loaded {len(matches)} Premier League matches')
        
        # Extract team data from matches
        for match in matches[:10]:  # Limit to first 10 matches for demo
            home_team = match.get('home_team', {})
            away_team = match.get('away_team', {})
            
            # Store team data
            if home_team.get('home_team_name'):
                team_data[home_team['home_team_name']] = {
                    'id': home_team.get('home_team_id'),
                    'name': home_team['home_team_name']
                }
            if away_team.get('away_team_name'):
                team_data[away_team['away_team_name']] = {
                    'id': away_team.get('away_team_id'),
                    'name': away_team['away_team_name']
                }
            
            # Store match data
            match_data.append({
                'match_id': match.get('match_id'),
                'home_team': home_team.get('home_team_name', 'Unknown'),
                'away_team': away_team.get('away_team_name', 'Unknown'),
                'home_score': match.get('home_score', 0),
                'away_score': match.get('away_score', 0),
                'match_date': match.get('match_date', 'Unknown')
            })
            
            # Fetch lineups for player data
            try:
                match_id = match.get('match_id')
                lineups_url = f'https://raw.githubusercontent.com/statsbomb/open-data/master/data/lineups/{match_id}.json'
                with urllib.request.urlopen(lineups_url) as lineup_response:
                    lineups = json.loads(lineup_response.read())
                    
                    for lineup in lineups:
                        team_name = lineup.get('team_name', 'Unknown')
                        for player in lineup.get('lineup', []):
                            player_id = player.get('player_id')
                            player_name = player.get('player_name')
                            if player_id and player_name:
                                player_data[player_id] = {
                                    'name': player_name,
                                    'team': team_name,
                                    'positions': player.get('positions', [])
                                }
            except Exception:
                pass  # Continue if lineup data not available
        
        print(f'✓ Loaded data for {len(team_data)} teams')
        print(f'✓ Loaded data for {len(player_data)} players')
        print(f'✓ Loaded {len(match_data)} match records')
        
        statsbomb_data = {
            'players': player_data,
            'teams': team_data,
            'matches': match_data
        }
        
except Exception as e:
    print(f'⚠ StatsBomb data not available: {str(e)}')
    print(f'✓ Using simulated data only')
    statsbomb_data = None


## 10. Generate Training Data

Simulate multiple matches to generate training data.

In [ ]:
all_events = []

print('Simulating matches...')
for match_id in range(5):
    team1 = create_team(1, '433')
    team2 = create_team(2, '433')
    
    simulator = MatchSimulator(team1, team2)
    events = simulator.simulate_match(num_actions=150)
    all_events.extend(events)
    
    print(f'  Match {match_id + 1}: {len(events)} events')

print(f'✓ Generated {len(all_events)} total events from {5} matches')
print(f'Success rate: {sum(1 for e in all_events if e.success) / len(all_events):.1%}')

## 11. ML Model

Train a Random Forest model for tactical recommendations.

In [ ]:
def prepare_ml_data(events: List[MatchEvent]):
    """Prepare features and labels for ML training."""
    X = []
    y = []
    
    for event in events:
        features = [
            event.position.x,
            event.position.y,
            1 if event.player_role == Role.DEF else (2 if event.player_role == Role.MID else 3),
            list(TacticalPattern).index(event.pattern),
            event.distance
        ]
        X.append(features)
        y.append(1 if event.success else 0)
    
    return np.array(X), np.array(y)

X, y = prepare_ml_data(all_events)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Training Random Forest model...')
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'✓ Model trained')
print(f'Accuracy: {accuracy:.3f}')
print(f'Training samples: {len(X_train)}, Test samples: {len(X_test)}')

## 12. Visualizations

Create comprehensive visualizations of the match data.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

action_counts = Counter([e.action.value for e in all_events])
axes[0, 0].bar(action_counts.keys(), action_counts.values(), color='steelblue')
axes[0, 0].set_title('Action Distribution', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Count')
axes[0, 0].tick_params(axis='x', rotation=45)

action_success = defaultdict(list)
for e in all_events:
    action_success[e.action.value].append(e.success)
success_rates = {k: np.mean(v) for k, v in action_success.items()}
axes[0, 1].bar(success_rates.keys(), success_rates.values(), color='forestgreen')
axes[0, 1].set_title('Success Rate by Action', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Success Rate')
axes[0, 1].set_ylim([0, 1])
axes[0, 1].tick_params(axis='x', rotation=45)

pattern_counts = Counter([e.pattern.value for e in all_events])
axes[0, 2].pie(pattern_counts.values(), labels=pattern_counts.keys(), autopct='%1.1f%%',
               colors=['#ff9999', '#66b3ff', '#99ff99', '#ffcc99'])
axes[0, 2].set_title('Tactical Pattern Distribution', fontsize=14, fontweight='bold')

pass_events = [e for e in all_events if e.pass_type is not None]
if pass_events:
    pass_type_counts = Counter([e.pass_type.value for e in pass_events])
    axes[1, 0].bar(pass_type_counts.keys(), pass_type_counts.values(), color='coral')
    axes[1, 0].set_title('Pass Type Distribution', fontsize=14, fontweight='bold')
    axes[1, 0].set_ylabel('Count')
    axes[1, 0].tick_params(axis='x', rotation=45)

x_coords = [e.position.x for e in all_events]
y_coords = [e.position.y for e in all_events]
axes[1, 1].hexbin(x_coords, y_coords, gridsize=20, cmap='YlOrRd', mincnt=1)
axes[1, 1].set_title('Action Heatmap', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('X Position')
axes[1, 1].set_ylabel('Y Position')
axes[1, 1].set_xlim([0, 105])
axes[1, 1].set_ylim([0, 68])

if pass_events:
    risk_by_type = {}
    outcome_by_type = {}
    for pt in PassType:
        type_passes = [e for e in pass_events if e.pass_type == pt]
        if type_passes:
            risk_by_type[pt.value] = PASS_CONFIGS[pt].risk_factor
            outcome_by_type[pt.value] = np.mean([e.success for e in type_passes])
    
    axes[1, 2].scatter(list(risk_by_type.values()), list(outcome_by_type.values()), 
                      s=200, alpha=0.6, c=range(len(risk_by_type)), cmap='viridis')
    for i, txt in enumerate(risk_by_type.keys()):
        axes[1, 2].annotate(txt, (list(risk_by_type.values())[i], list(outcome_by_type.values())[i]),
                           fontsize=9, ha='center')
    axes[1, 2].set_title('Risk vs Success by Pass Type', fontsize=14, fontweight='bold')
    axes[1, 2].set_xlabel('Risk Factor')
    axes[1, 2].set_ylabel('Success Rate')
    axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('football_tactics_analysis.png', dpi=150, bbox_inches='tight')
print('✓ Visualizations created and saved')
plt.show()

## 13. Tactical Recommendations

Generate tactical recommendations based on game state.

In [ ]:
def get_tactical_recommendation(x: float, y: float, role: Role) -> Dict:
    """Get tactical recommendation for a given position and role."""
    role_encoding = 1 if role == Role.DEF else (2 if role == Role.MID else 3)
    
    recommendations = []
    for pattern_idx, pattern in enumerate(TacticalPattern):
        for distance in [10, 20, 40]:
            features = np.array([[x, y, role_encoding, pattern_idx, distance]])
            success_prob = model.predict_proba(features)[0][1]
            recommendations.append({
                'pattern': pattern.value,
                'distance': distance,
                'success_prob': success_prob
            })
    
    best = max(recommendations, key=lambda x: x['success_prob'])
    return best

print('\n=== Tactical Recommendations ===')
test_scenarios = [
    (25, 34, Role.DEF, 'Defensive position - center'),
    (52, 20, Role.MID, 'Midfield - left side'),
    (80, 34, Role.FWD, 'Attacking position - center'),
    (70, 10, Role.FWD, 'Wing position - left'),
    (40, 55, Role.MID, 'Midfield - right side')
]

for x, y, role, desc in test_scenarios:
    rec = get_tactical_recommendation(x, y, role)
    print(f'\n{desc}')
    print(f'  Position: ({x:.0f}, {y:.0f}), Role: {role.value}')
    print(f'  Recommended: {rec["pattern"]} with {rec["distance"]}m pass')
    print(f'  Success probability: {rec["success_prob"]:.1%}')

## 14. Save Model

Save the trained model and training data for future use.

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)
print('✓ Model saved to model.pkl')

training_data = {
    'events': [{
        'time': int(e.time),
        'action': e.action.value,
        'role': e.player_role.value,
        'x': float(e.position.x),
        'y': float(e.position.y),
        'pass_type': e.pass_type.value if e.pass_type else None,
        'pattern': e.pattern.value,
        'success': bool(e.success),
        'distance': float(e.distance)
    } for e in all_events],
    'metadata': {
        'total_events': len(all_events),
        'num_matches': 5,
        'model_accuracy': float(accuracy)
    }
}

with open('training_data.json', 'w') as f:
    json.dump(training_data, f, indent=2)
print('✓ Training data saved to training_data.json')

print('\n=== Summary Statistics ===')
print(f'Total events: {len(all_events)}')
print(f'Overall success rate: {sum(1 for e in all_events if e.success) / len(all_events):.1%}')
print(f'Model accuracy: {accuracy:.1%}')
print(f'Pass events: {len([e for e in all_events if e.action == ActionType.PASS])}')
print(f'Shot events: {len([e for e in all_events if e.action == ActionType.SHOT])}')
print('\n✓ Football Tactics AI implementation complete!')